In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
import datetime
import openpyxl
from colorama import Fore, Back, Style #print colored text
from time import sleep
import os

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36 Edg/88.0.705.56',
    'Cookie': '_last_loginuname=viviyanwei; Hm_lvt_e14a1f00255ca6d026803cca20b08490=1607743514; href=https%3A%2F%2Finfo.mybxg.com%2Fm%2F20%2F1210%2F16%2F91D67724C337E319.html; accessId=5d36a9e0-919c-11e9-903c-ab24dbab411b; qimo_xstKeywords_5d36a9e0-919c-11e9-903c-ab24dbab411b=; _login_token=3191c6adf1eb4ce191bcbc278dc29301; _login_mid=2468603; _login_uid=1626926; _login_ip=116.228.72.50; 3191c6adf1eb4ce191bcbc278dc29301=11%3D5%261%3D10; Hm_lpvt_e14a1f00255ca6d026803cca20b08490=1607926822; qimo_seosource_5d36a9e0-919c-11e9-903c-ab24dbab411b=%E5%85%B6%E4%BB%96%E7%BD%91%E7%AB%99; qimo_seokeywords_5d36a9e0-919c-11e9-903c-ab24dbab411b=%E6%9C%AA%E7%9F%A5; pageViewNum=9',
}
# User-Agent获取：打开edge浏览器，在地址栏输入edge://version/ 里面就有user agent的信息
# Cookie获取：用户名密码可以找李忆君(Yvonne)要，打开我的钢铁网首页，先登录，然后获取cookie可以参考这个帖子https://blog.csdn.net/MuWinter/article/details/75313476

In [14]:
#用来计算运行时间
program_start_time = datetime.datetime.now()

In [2]:
file_path = r"C:\Users\cnsghlubo\Downloads\我的钢铁网" #文件夹目录
files = os.listdir(file_path) #获得文件夹中所有文件的名称列表
excel_files = [file[-15:-5] for file in files if file[-4:] == 'xlsx'] #如果文件为excel格式，获取文件名中的日期，即倒数第15至倒数第6的字符
date_list = [] #全部符合规则的日期
for i in excel_files:
    try:
        date = datetime.datetime.strptime(i, '%Y-%m-%d') #将日期字符转换为datetime格式，用于比较大小
        date_list.append(date)
    except:
        pass
last_date = max(date_list).date() #选择文件夹中最新的日期

In [3]:
last_date

datetime.date(2021, 4, 19)

In [17]:
start_date = last_date + datetime.timedelta(days = 1)
end_date = datetime.date.today() + datetime.timedelta(days = 1) #日期范围为start_date到end_date-1，不包含end_date当天的日期
page_range = 1 #选择页数
local_path = str(start_date) + '_to_' + str(end_date - datetime.timedelta(days=1)) + '_on_' + str(datetime.date.today()) + '.xlsx' #如果保存路径与ipynb文件的地址一致，则只需要明确文件名，不需要路径

"""
将原材料分为2组，
dict_urls_1 在网站中的表格有 table header，即'th'
dict_urls_2 无 table header
"""
dict_urls = {
    'dict_urls_1': { 
    '无锡300系不锈钢冷轧卷板': 'https://list.info.mybxg.com/market/p-665,666,667,2222,2257,2259------0-01010301-------1.html', #原来叫‘无锡300系冷轧卷板’
    '无锡300系不锈钢热轧卷板': 'https://list.info.mybxg.com/market/p-671-0----0102160102-0-01010301-------1.html', #原来叫‘无锡300系热轧卷板’
    '300系不锈钢工业无缝管': 'https://list.info.mybxg.com/market/p-681-----0102160203-0--------1.html',
    '300系不锈钢工业焊管': 'https://list.info.mybxg.com/market/p-678-----010216020201-0--------1.html',
    '不锈钢角钢': 'https://list.info.mybxg.com/market/p-691-----0102160303-0--------1.html'
    },
    'dict_urls_2': { 
    '上海市场工角槽钢': 'https://list1.mysteel.com/market/p-227-0----01010701-0-010101-------1.html',
    '上海市场冷轧板卷': 'https://list1.mysteel.com/market/p-221-----01010401-0-010101-------1.html',
    '上海市场热轧板卷': 'https://list1.mysteel.com/market/p-221-----01010301-0-010101-------1.html',
    '上海市场低合金卷': 'https://list1.mysteel.com/market/p-221-----01010302-0-010101-------1.html',
    '上海市场镀锌板卷': 'https://list1.mysteel.com/market/p-221-----01010501-0-010101-------1.html',
    '上海市场无缝管': 'https://list1.mysteel.com/market/p-221-----01010901-0-010101-------1.html',
    '沈阳市场H型钢': 'https://list1.mysteel.com/market/p-223-----01010705-0-01050101-------1.html',
    '上海市场方矩管': 'https://list1.mysteel.com/market/p-221-----01010904-0-010101-------1.html',
    '长春市场工角槽钢': 'https://list1.mysteel.com/market/p-223-----01010701-0-01050201-------1.html',
    '长春市场冷轧板卷': 'https://list1.mysteel.com/market/p-223-----01010401-0-01050201-------1.html',
    '烟台市场热轧开平板': 'https://list1.mysteel.com/market/p-223-----01010306-0-01010205-------1.html'
    }
}

dict_urls_1_list = list(dict_urls['dict_urls_1'].items())
dict_urls_2_list = list(dict_urls['dict_urls_2'].items())
local_path

'2021-04-13_to_2021-04-14_on_2021-04-14.xlsx'

In [73]:
#没用上，可以参考
def datetime_str_to_date (date_time_str):
    """
    convert dateimte str to dateimte to date
    input '%Y-%m-%d %H:%M'
    output %Y-%m-%d
    """
    try:
        date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M')
        date_ojb = date_time_obj.date()
    except TypeError:
        print('Type error!')
    else:
        return date_ojb

In [74]:
def get_n_pages_url (page1_url, n):
    """
    input first page url ending with '1.html'
    output list of url of n pages
    """
    page_n_list = []
    for i in range(1, n + 1):
        page_n_url = page1_url.replace('1.html', str(i) + '.html')
        page_n_list.append(page_n_url)
    return page_n_list

In [75]:
def get_title_info (input_page_list):
    """
    input a list of page url
    output a list of tile info 即（日期，title，链接）
    """
    page_list = []
    for url in input_page_list:
        response = requests.get(url)
        content = response.content
        soup = BeautifulSoup(content, "lxml")

        all_titles = soup.findAll('li', {"data-val":"0"})

        for i in range(len(all_titles)):
            txt = all_titles[i]
            url = 'https:' + txt.a.get('href')
            title = txt.a.get('title')
            date = datetime.datetime.strptime(txt.span.get_text(), '%Y-%m-%d %H:%M')
            page_list.append([title, date, url])

    return page_list

In [76]:
"""dict1"""
for d1_item in dict_urls_1_list:
#     sleep(3) #在每次访问之间暂停几秒，防止网站访问的request被拒绝
    pages = get_n_pages_url(d1_item[1], n = page_range) #创建list，包含所有页数的链接
    titles = get_title_info(pages)
    title_list_filtered = [] #append 所有页数中标题的链接
    for i in range(len(titles)):
        if d1_item[0] in titles[i][0]: #筛选包含原材料名称的标题链接
            title_list_filtered.append(titles[i])
    print(Fore.YELLOW, d1_item) #print title
    df_list = [] #创建空list，用于后续将一个原材料的所有数据存入，并保存为dataframe
    try:
        for item in title_list_filtered: 
            title = item[0] #提取title_list_filtered中每一个item的title，date和链接
            date = item[1]
            url = item[2]

            response = requests.get(url, headers = headers)
            content = response.content
            soup = BeautifulSoup(content,'lxml') #解析网页数据

            table_header = soup.findAll('th')
            header_list = []
            for i in table_header:
                text = i.get_text()
                text_strip = text.strip('\r\n\t请筛选').strip()
                header_list.append(text_strip)

            table_body = soup.find('tbody')
            table_row = table_body.findAll('tr')
            for tr in table_row:
                td = tr.findAll('td')
                row_list = []
                for i in td:
                    text = i.get_text()
                    text_strip = text.strip('\r\n\t请筛选').strip()
                    row_list.append(text_strip)
                df_list.append([date,title, url] + row_list[:7]) #保留网页中表格的前7列，加上之前提取的title，date，链接3列

        df = pd.DataFrame(data = df_list, columns = ['日期','标题','网址'] + header_list[:7]) #保留header list的3+7列，与网页数据一致
        df = df[(df['日期'] >= str(start_date)) & (df['日期'] < str(end_date))] #日期筛选
        df = df.astype({str(np.array(df.columns)[df.columns.str.contains(pat = '价格')][0]): int}) #价格列数据类型转换为数值
        print(Fore.BLACK, df)
        if d1_item[0] == dict_urls_1_list[0][0]:
            writer = pd.ExcelWriter(local_path, mode = 'w', engine = 'openpyxl')
        else:
            writer = pd.ExcelWriter(local_path, mode = 'a', engine = 'openpyxl')
        df.to_excel(writer, index = False, header = None, sheet_name = d1_item[0], encoding = 'utf-8') #从已有数据的下一行开始写入新数据
        writer.save()
        writer.close()
    except:
        print(Fore.RED, d1_item)

 ('无锡300系不锈钢冷轧卷板', 'https://list.info.mybxg.com/market/p-665,666,667,2222,2257,2259------0-01010301-------1.html')
                      日期                   标题  \
267 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
268 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
269 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
270 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
271 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
..                  ...                  ...   
529 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
530 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
531 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
532 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   
533 2021-04-09 09:47:00  9日无锡300系不锈钢冷轧卷板价格行情   

                                                    网址  钢种             规格  \
267  https://info.mybxg.com/m/21/0409/09/262AB152DE...  卷板     0.4*1219*C   
268  https://info.mybxg.com/m/21/0409/09/262AB152DE...  卷板     0.5*1219*C   
269  https://info.mybxg.com/m/21/0409/09/262AB152DE...  卷板     0.6*1219*C   

In [77]:
"""dict2"""
for d2_item in dict_urls_2_list:
#     sleep(3) #pause between requests
    pages = get_n_pages_url(d2_item[1], n = page_range)
    titles = get_title_info(pages)
    title_list_filtered = []
    for i in range(len(titles)):
        if d2_item[0] in titles[i][0]:
            title_list_filtered.append(titles[i])
    print(Fore.YELLOW, d2_item) #print title
    try:
        df_list = []
        for item in title_list_filtered:
            title = item[0]
            date = item[1]
            url = item[2]

            response = requests.get(url, headers = headers)
            content = response.content
            soup = BeautifulSoup(content,'lxml')

            table_body = soup.find('table',{'id':'marketTable'})
            table_row = table_body.findAll('tr')
            table_list=[]
            for tr in table_row:
                td = tr.findAll('td')
                row_list = []
                for i in td:
                    text = i.get_text()
                    text_strip = text.strip('\r\n\t请筛选').strip()
                    row_list.append(text_strip)
        #         df_list.append([date,title, url] + row_list[:7])
                table_list.append([date,title, url] + row_list[:7])
            header_list = table_list.pop(0) #剪切第一行，作为header
            table_list.pop(0)
            df_list.extend(table_list)
        df = pd.DataFrame(data = df_list, columns = ['日期','标题','网址'] + header_list[3:10]) #保留3+7列
        df = df[(df['日期'] >= str(start_date)) & (df['日期'] < str(end_date))]
        df = df.astype({str(np.array(df.columns)[df.columns.str.contains(pat = '价格')][0]): int}) #价格列数据类型转换为数值
        print(Fore.BLACK, df)
        writer = pd.ExcelWriter(local_path, mode = 'a', engine = 'openpyxl')
        df.to_excel(writer, index = False, header = None, sheet_name = d2_item[0], encoding = 'utf-8')
        writer.save()
        writer.close()
    except:
        print(Fore.RED, d2_item)

 ('上海市场工角槽钢', 'https://list1.mysteel.com/market/p-227-0----01010701-0-010101-------1.html')
                      日期              标题  \
70  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
71  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
72  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
73  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
74  2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
..                  ...             ...   
135 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
136 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
137 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
138 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   
139 2021-04-09 10:41:00  9日上海市场工角槽钢价格行情   

                                                    网址    品名   规格（mm）     材质  \
70   https://xinggang.mysteel.com/m/21/0409/10/EFDD...    角钢  30*30*3  Q235B   
71   https://xinggang.mysteel.com/m/21/0409/10/EFDD...    角钢  40*40*4  Q235B   
72   https://xinggang.mysteel.com/m/21/0409/10/EFDD...    角钢  50*50*5  Q235B   
73   https://xinggang.mysteel.com/m/21/0409/10/EFDD...    角钢  50*50*5 

In [78]:
program_end_time = datetime.datetime.now()
print('总计运行时间: {}'.format(program_end_time - program_start_time))

总计运行时间: 0:06:10.141739


In [3]:
!jupyter nbconvert --to script MySteel-v5.ipynb

[NbConvertApp] Converting notebook MySteel-v5.ipynb to script
[NbConvertApp] Writing 10580 bytes to MySteel-v5.py


In [ ]:
# !pip install pyinstaller

In [4]:
!pyinstaller --onefile MySteel-v5.py

3295 INFO: PyInstaller: 4.2
3295 INFO: Python: 3.8.8 (conda)
3295 INFO: Platform: Windows-10-10.0.18362-SP0
3295 INFO: wrote C:\Users\cnsghlubo\Downloads\我的钢铁网\MySteel-v5.spec
3302 INFO: UPX is not available.
3302 INFO: Extending PYTHONPATH with paths
['C:\\Users\\cnsghlubo\\Downloads\\我的钢铁网',
 'C:\\Users\\cnsghlubo\\Downloads\\我的钢铁网']
3380 INFO: checking Analysis
3380 INFO: Building Analysis because Analysis-00.toc is non existent
3380 INFO: Initializing module dependency graph...
3390 INFO: Caching module graph hooks...
3403 WARNING: Several hooks defined for module 'win32ctypes.core'. Please take care they do not conflict.
3413 INFO: Analyzing base_library.zip ...
13970 INFO: Processing pre-find module path hook distutils from 'c:\\users\\cnsghlubo\\anaconda3\\lib\\site-packages\\PyInstaller\\hooks\\pre_find_module_path\\hook-distutils.py'.
13970 INFO: distutils: retargeting to non-venv dir 'c:\\users\\cnsghlubo\\anaconda3\\lib'
22162 INFO: Caching module dependency graph...
22431 I